# n-gram language model

Let's write a class for our language model with an end-to-end pipeline inside of it: parsing the text file, text preprocessing, computing the frequency counts, creating the transition matrix, random text generation.

In [1]:
import numpy as np
from typing import *
from string import punctuation
from nltk.tokenize import sent_tokenize
from collections import Counter, defaultdict

punctuation += '«»–—…“”'

class NgramLanguageModel:
    def __init__(self, path: str, n: int = 2) -> None:
        if n < 2:
            # Strictly speaking, it's possible, but I've decided not to deal with unigram language models here.
            raise Exception('Parameter N of an n-gram language model cannot be less than 2!')
        self.n = n
        with open(path, encoding='utf-8') as f:
            self.text = f.read()
        self.sentences = map(lambda sentence: ['<START>'] * (self.n - 1) + \
                             [x for x in self.tokenize(sentence) if x] + ['<END>'],
                             sent_tokenize(self.text))
        self.token2id = {}
        if self.n > 2:
            self.ngram2id = {}
        self.ngrams = [Counter(), Counter()]
        self.get_ngram_counts()
        self.vocab_size = len(self.token2id)
        self.id2token = {value: key for key, value in self.token2id.items()}
        self.matrix = np.zeros((len(self.ngrams[0]), self.vocab_size))
        if self.n > 2:
            self.id2ngram = {value: key for key, value in self.ngram2id.items()}
        self.populate_matrix()
    
    def get_ngram_counts(self) -> None:
        if self.n == 2:
            for sentence in self.sentences:
                self.ngrams[0].update(sentence)
                self.ngrams[1].update(list(self.ngrammer(sentence, 2)))
                self.update_token2id(sentence)
        if self.n > 2:
            for sentence in self.sentences:
                ngrams = list(self.ngrammer(sentence, self.n-1))
                self.ngrams[0].update(ngrams)
                self.ngrams[1].update(list(self.ngrammer(sentence, self.n)))
                self.update_token2id(sentence)
                self.update_ngram2id(ngrams)
    
    def update_token2id(self, tokens: List[str]) -> None:
        for token in tokens:
            if token not in self.token2id:
                self.token2id[token] = len(self.token2id)
    
    def update_ngram2id(self, ngrams: List[str]) -> None:
        for ngram in ngrams:
            if ngram not in self.ngram2id:
                self.ngram2id[ngram] = len(self.ngram2id)
    
    def populate_matrix(self):
        if self.n == 2:
            for ngram, count in self.ngrams[1].items():
                src, dest = ngram.split()
                self.matrix[self.token2id[src], self.token2id[dest]] = \
                    count / self.ngrams[0][src]
        if self.n > 2:
            for ngram, count in self.ngrams[1].items():
                ngram_splitted = ngram.split()
                src = ' '.join(ngram_splitted[:-1])
                dest = ngram_splitted[-1]
                self.matrix[self.ngram2id[src], self.token2id[dest]] = \
                    count / self.ngrams[0][src]

    def generate_random_text(self, n: int = 100,
                             random_state: int = None) -> Generator[str, None, None]:
        if self.n == 2:
            current_idx = self.token2id['<START>']
            for i in range(n):
                if random_state:
                    np.random.seed(random_state)
                chosen = np.random.choice(self.vocab_size, p=self.matrix[current_idx])
                yield self.id2token[chosen]
                if chosen == self.token2id['<END>']:
                    chosen = self.token2id['<START>']
                current_idx = chosen
        if self.n > 2:
            current_idx = self.ngram2id[' '.join(['<START>'] * (self.n-1))]
            for i in range(n):
                if random_state:
                    np.random.seed(random_state)
                chosen = np.random.choice(self.vocab_size, p=self.matrix[current_idx])
                yield self.id2token[chosen]
                if chosen == self.token2id['<END>']:
                    current_idx = self.ngram2id[' '.join(['<START>'] * (self.n-1))]
                else:
                    current_idx = self.ngram2id[' '.join(self.id2ngram[current_idx].split()[1:] +
                                                         [self.id2token[chosen]])]
            
    @staticmethod
    def tokenize(sentence: str) -> map:
        return map(lambda x: x.strip(punctuation).replace('ё', 'е'),
                   sentence.strip().lower().split())
    
    @staticmethod
    def ngrammer(sentence: List[str], n: int) -> Generator[str, None, None]:
        for i in range(len(sentence)-n+1):
            yield ' '.join(sentence[i: i+n])

# Bigram language model

In [2]:
tolstoy2 = NgramLanguageModel('anna_karenina.txt', 2)
dostoevsky2 = NgramLanguageModel('besy_dostoevsky.txt', 2)

In [3]:
print(' '.join([x for x in tolstoy2.generate_random_text()]).replace('<END>', '\n'))

и пожить там сиял своею внешностью продолжал он скажет кити и старался левин уже уехали 
 приезжайте вечером ни наука на пюпитре требуют от развода при ней при каждом его василий лукич был намерен был совершенно доволен своею внешностью поддерживал всякий согласится что последняя канавка с теми мыслями о прекрасно 
 ты не нравилось 
 если натура слишком оживленный с полным ртом поднял красно-пегого теленка на веселую какою она покраснела и не думаю что я считаю его шаги 
 а брал все приготовил это 
 старуха укутывала ноги рыжего направо кто был влюблен в назначенный день 
 взойдя наверх чтобы


In [4]:
print(' '.join([x for x in dostoevsky2.generate_random_text()]).replace('<END>', '\n'))

ей сами это от ури не отвечаете 
 у нас 
 теперь 
 если правда все больше врагов я ухожу не помня себя свою круглую совсем вон у a sonn&#233 
 эта блондинка с оглавлениями указаниями с визитом 
 да что ее обнял страх пред вами с ничем и ангел она принуждена петь с обильно заваренным чаем 
 лет семь человек 
 николай всеволодович слушал сдвинув брови были тщеславны до колена в спасов 
 именно пробормотал кириллов объявляю наш русский атеизм никогда прежде и отпер все страшно 
 за что ничего а в зеркало 
 вы только которые посетитель подымался


# Trigram language model

In [5]:
tolstoy3 = NgramLanguageModel('anna_karenina.txt', 3)
dostoevsky3 = NgramLanguageModel('besy_dostoevsky.txt', 3)

In [6]:
print(' '.join([x for x in tolstoy3.generate_random_text()]).replace('<END>', '\n'))

она знала шел к ней эта атмосфера забот которою он любовался на эту тему прекратился 
 она не поднялась и оттолкнула его от этого выйти что она беременна 
 эти точно такие же и на восторженные ее уверения в нежности его теперь она была а была долли с вронским совсем конченным что оно было вне того круга в котором теперь чувствовала себя виноватою 
 забыл все то что увязили его лошадей и когда пошел искать убитого то не следует а то этак можно самому привыкнуть и ее закопают и федора подавальщика с его красотой милою улыбкой и держась своей первой


In [7]:
print(' '.join([x for x in dostoevsky3.generate_random_text()]).replace('<END>', '\n'))

гм 
 мучь меня казни меня срывай на мне злобу вскричал он с чрезвычайным замешательством и краснея как только можно вообразить себе 
 258 я совсем другое дело настоящий пожар тут ужас и все я думаю даже что он вовсе не выходить а сейчас же объясниться 
 сплетник 
 но ведь тогда что они не обращали на него 
 наконец разместились утихла и музыка 
 ведь он знает это заранее все равно 
 вдруг и брякнет мне главный вопрос который вас интересует пробормотал ставрогин 
 вы атеист потому что ставрогин не может вы кликните 
 в последние минуты не все


The predictions of a trigram language model better resemble a coherent text. Perhaps, if we keep increasing the n-gram size, the quality of the generated text will keep growing, but the problem of training corpus size will keep growing as well. The longer the context we want to consider is, the more data we need. Besides, the computed frequency counts have to be smoothed in some tricky ways for a less predetermined output, but this is outside the scope of this homework.